# Deconvolução de Euler aplicada na anomalia de campo total produzida por uma esfera

Este código utiliza a biblioteca de modelagem e inversão de dados geofísicos denominada [Fatiando a Terra](http://www.fatiando.org/index.html).

### Coisas para importar

In [1]:
%matplotlib inline
import numpy as np
from fatiando import mesher, gridder, utils
from fatiando.gravmag import sphere
from fatiando.vis import mpl

### Inclinação e a declinação do campo geomagnético local

In [2]:
inc = 56. # graus
dec = -60. # graus
amp = 23500.0 # nT

In [3]:
F = utils.ang2vec(amp, inc, dec) # vetor campo geomagnético local

In [4]:
print F

[  6570.51661578 -11380.46861051  19482.38295504]


### Modelo de corpo esférico

In [5]:
model = [mesher.Sphere(0., 0., 700., 500., {'magnetization': utils.ang2vec(10., inc, dec)})]

### Pontos onde são calculados os dados

In [11]:
shape = (100, 100) # número de pontos ao longo de x e y
N = shape[0]*shape[1] # número de pontos
area = (-5000, 5000, -5000, 5000) # x mínimo, x máximo, y mínimo e y máximo da área
xp, yp, zp = gridder.regular(area, shape, z=-100) # grade regular localizada em z = -100 m

### Anomalia de campo total (em nT)

In [7]:
act = sphere.tf(xp, yp, zp, model, inc, dec)

### Derivadas espaciais da anomalia de campo total (em nT/m)

In [8]:
delta = 10.

dx_act = (sphere.tf(xp + delta, yp, zp, model, inc, dec) - \
          sphere.tf(xp - delta, yp, zp, model, inc, dec))/(2.*delta)

dy_act = (sphere.tf(xp, yp + delta, zp, model, inc, dec) - \
          sphere.tf(xp, yp - delta, zp, model, inc, dec))/(2.*delta)

dz_act = (sphere.tf(xp, yp, zp + delta, model, inc, dec) - \
          sphere.tf(xp, yp, zp - delta, model, inc, dec))/(2.*delta)

### Deconvolução de Euler

In [44]:
eta = 3.0

In [45]:
y = np.reshape(xp*dx_act + yp*dy_act + zp*dz_act + eta*act, (N, 1))
A = np.hstack((np.reshape(dx_act, (N, 1)), 
               np.reshape(dy_act, (N, 1)), 
               np.reshape(dz_act, (N, 1)), 
               np.zeros((N,1)) + eta))

In [46]:
x = np.linalg.solve(np.dot(A.T, A), np.dot(A.T, y))

In [47]:
print 'x: %12.5f | %12.5f' % (model[0].x,x[0])
print 'y: %12.5f | %12.5f' % (model[0].y,x[1])
print 'z: %12.5f | %12.5f' % (model[0].z,x[2])

x:      0.00000 |     -0.00088
y:      0.00000 |     -0.00013
z:    700.00000 |    699.85154
